### **API SADVR - Portrait statistique**  
https://www.cen.umontreal.ca/espacedoc/sadvr/  

Ce NoteBook est destiné à l'extraction et la visualisation de statistiques relatives aux professeur·e·s et à leurs expertises à partir de l'API de la vitrine de la recherche (SADVR). 
Celles-ci seront intégrées en un tableau de bord [PowerBI](https://wiki.umontreal.ca/display/SIE/Power+BI) permettant d'avoir un portrait d'ensemble des données.  

---

In [8]:
import pandas as pd
import requests 
import json

from utils.sadvr_utils import *
import plotly.express as px
import plotly.graph_objects as go

from rdflib import *

data = updateInfoProfs()
expertises = data[['idsadvr', 'expertise']]

departements = getTable('individus')[['idsadvr', 'uniteAdmin']]

expertises = expertises.merge(departements, on='idsadvr')

def uniteAdminDepartement(t: str):
    liste =  t.split(' - ')
    if len(liste) >1:
        return liste[1]
    else:
        return liste[0]

expertises['département'] = expertises['uniteAdmin'].astype(str).apply(uniteAdminDepartement)
expertises = expertises.drop(columns='uniteAdmin')


## Normalisation des données
toNormalize = ['expertise', 'expertise.disciplines']
for c in toNormalize:
    expertises = explodeNormalize(expertises, c)

expertises = expertises.dropna(subset = 'expertise.disciplines.uid') 
expertises = expertises[expertises['expertise.motsCles'].astype(str) != '[]']
expertises = expertises[expertises['département'].astype(str) != 'None']

expertises = explodeNormalize(expertises, 'expertise.motsCles')

expertises = expertises[['idsadvr', 'département',
      'expertise.disciplines.uid', 'expertise.disciplines.codeLangue','expertise.disciplines.nom', 
       'expertise.motsCles.uid', 'expertise.motsCles.nom', 'expertise.motsCles.codeLangue']]

expertises = expertises.sort_values(by=[
    f'expertise.disciplines.uid', f'expertise.disciplines.codeLangue',
    f'expertise.motsCles.uid', f'expertise.motsCles.codeLangue'], ascending=[True, False, True, False])

expertises = expertises.drop_duplicates(subset=['idsadvr', 'expertise.motsCles.uid', 'expertise.disciplines.uid'])
expertises = expertises.drop(columns=['expertise.disciplines.codeLangue', 'expertise.motsCles.codeLangue'])

## Extraire les fréquences associées aux disciplines et aux mots-clés: elles vont permettre d'assigner
# une taille aux noeuds dans le graphe (plus fréquent = plus gros )
def freqVariableExpertises(variable: str, df: pd.DataFrame = expertises) -> pd.DataFrame:
    output = df[['idsadvr', f'expertise.{variable}.nom', 
                       f'expertise.{variable}.uid']].dropna(subset=f'expertise.{variable}.uid').drop_duplicates()

    output = output.groupby([f'expertise.{variable}.nom', f'expertise.{variable}.uid'])['idsadvr'].count().reset_index().rename(columns={'idsadvr': 'count'})
    output = output[[f'expertise.{variable}.nom', 'count']]

    return output

# Disciplines
freqDisciplines = freqVariableExpertises('disciplines')

# Mots-clés
freqMotsCles = freqVariableExpertises('motsCles')

# Départements
freqDepartements = expertises[['idsadvr', 'département']].drop_duplicates(subset='idsadvr')
freqDepartements = freqDepartements.groupby('département')['idsadvr'].count().reset_index().rename(columns={'idsadvr':'count'})

**Expertises de recherche: cartographie des expertises par mots-clés**

Départements / Disciplines / Mots-clés 

Voir si on peut extraire un graphe et le visualiser

In [16]:
# premier exemple: Département de neurosciences
# Une fois que ça aura fonctionné pour un département, on va créer une fonction qui pourra le faire pour tous les départements
departements = expertises

departNeuro = departements[departements['département'] == 'Département de neurosciences']
departNeuro

,idsadvr,département,expertise.disciplines.uid,expertise.disciplines.nom,expertise.motsCles.uid,expertise.motsCles.nom
43518,in14277,Département de neurosciences,104,Optométrie,135,Dégénérescence cellulaire
43520,in14277,Département de neurosciences,104,Optométrie,164,Différenciation cellulaire
43522,in14277,Département de neurosciences,104,Optométrie,227,Génétique du vieillissement
43524,in14277,Département de neurosciences,104,Optométrie,234,Génome
43526,in14277,Département de neurosciences,104,Optométrie,243,Greffes neuronales et cellules souches
...,...,...,...,...,...,...
128680,in17966,Département de neurosciences,98,Neurosciences,98,Communication neuronale et neurotransmission
165428,in29442,Département de neurosciences,98,Neurosciences,98,Communication neuronale et neurotransmission
183508,in32497,Département de neurosciences,98,Neurosciences,98,Communication neuronale et neurotransmission
184990,in32848,Département de neurosciences,98,Neurosciences,98,Communication neuronale et neurotransmission


In [70]:


# Neurosciences: uid = 98
neuro = disciplines[disciplines['expertise.disciplines.uid'] == '98']

neuro = neuro.drop_duplicates(subset=['idsadvr', 'expertise.motsCles.uid'])
neuro = neuro [['idsadvr',	'expertise.disciplines.uid', 'expertise.disciplines.nom', 'expertise.motsCles.uid',
                'expertise.motsCles.nom']]

freqMotsCles = pd.DataFrame(plotVariable(neuro, 'expertise.motsCles.uid')).to_dict('records')
freqMotsCles = {x['labels'] : x['count'] for x in freqMotsCles}

neuro['expertises.motCles.count'] = neuro['expertise.motsCles.uid'].map(freqMotsCles)
neuro = neuro.drop_duplicates(subset = ['expertise.motsCles.uid']).drop(columns='idsadvr')

neuro = neuro.sort_values(by='expertises.motCles.count', ascending=False).reset_index(drop=True)

*On va essayer de créer un graphe avec*

In [ ]:
# Exemple de visualisation qui pourrait être utilisée avec le graphe permettant de cartographier les expertises de recherche

import networkx as nx
import plotly.graph_objects as go

# Create a NetworkX graph
G = nx.Graph()
G.add_nodes_from([1, 2, 3, 4, 5])

# Add edges
G.add_edges_from([(1, 2), (2, 3), (3, 4), (4, 5), (5, 1)])

# Define node sizes (you can replace this with your desired sizes)
node_sizes = [10, 20, 30, 40, 50]

# Create a Plotly figure
fig = go.Figure()

# Add nodes and edges to the Plotly figure
pos = nx.spring_layout(G)  # You can use a different layout algorithm
edge_trace = go.Scatter(x=[], y=[], line={'width': 0.5, 'color': '#888'})
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace['x'] += (x0, x1, None)
    edge_trace['y'] += (y0, y1, None)

node_trace = go.Scatter(
    x=[], y=[], text=[], mode='markers+text', hoverinfo='text',
    marker={'size': node_sizes, 'color': '#FF5733'}
)
for node in G.nodes():
    x, y = pos[node]
    node_trace['x'] += (x,)
    node_trace['y'] += (y,)

fig.add_trace(edge_trace)
fig.add_trace(node_trace)

# Customize layout to hide grid and remove axis
fig.update_layout(
    xaxis_visible=False,  # Hide x-axis
    yaxis_visible=False  # Hide y-axis
)

# Show the Plotly figure
fig.show()

In [21]:
expertises#.drop_duplicates()

,idsadvr,département,expertise.disciplines.uid,expertise.disciplines.nom,expertise.motsCles.uid,expertise.motsCles.nom
157272,in24090,Département de sciences économiques,1,Actuariat (gestion),3283,Mégadonnées
157274,in24090,Département de sciences économiques,1,Actuariat (gestion),5655,Économétrie financière
157276,in24090,Département de sciences économiques,1,Actuariat (gestion),5673,Finance
189218,in34368,Département d'anesthésiologie et de médecine d...,10,Anesthésiologie,112,"Conscience, subjectivité et identité personnelle"
189220,in34368,Département d'anesthésiologie et de médecine d...,10,Anesthésiologie,154,Développement cognitif
...,...,...,...,...,...,...
165428,in29442,Département de neurosciences,98,Neurosciences,98,Communication neuronale et neurotransmission
167260,in29687,Département de psychologie,98,Neurosciences,98,Communication neuronale et neurotransmission
183508,in32497,Département de neurosciences,98,Neurosciences,98,Communication neuronale et neurotransmission
184990,in32848,Département de neurosciences,98,Neurosciences,98,Communication neuronale et neurotransmission


In [ ]:
mappingDisciplines = pd.read_csv('tables/SADVR_disciplines.csv')
mappingDisciplines = mappingDisciplines[mappingDisciplines['noms.codeLangue'] == 'fre']
mappingDisciplines = {str(x['id']): x['noms.nom'] for x in mappingDisciplines.to_dict('records')}

mappingDepartements = pd.read_csv('tables/SADVR_departements.csv')
mappingDepartements = mappingDepartements[mappingDepartements['noms.codeLangue'] == 'fre']
mappingDepartements = {str(x['id']): x['noms.nom'] for x in mappingDepartements.to_dict('records')}


idDepart = "2353"
records = departNeuro.to_dict('records')


##### Définition du modèle de graphe
g = Graph()
g.bind('sadvr', 'https://www.recherche.umontreal.ca/vitrine/rest/api/1.7/umontreal/')

## Définition des classes
# Classe pour les départements
uriDepartements = URIRef(f'{baseURI}/departements/')

# Pour l'instant on a 1 département - à modifier quand ça fonctionne avec 1
uriDepartement = URIRef(f'{baseURI}/departements/{idDepart}')

g.add((uriDepartements, RDF.type, OWL.Class))
g.add((uriDepartements, RDFS.label, Literal('Département', lang='fr')))

# Classe pour les disciplines
uriDisciplines = URIRef(f'{baseURI}/disciplines/')

g.add((uriDisciplines, RDF.type, OWL.Class))
g.add((uriDisciplines, RDFS.label, Literal('Discipline', lang='fr')))

# Classe pour les mots-clés
uriMotsCles = URIRef(f'{baseURI}/motsCles/')

g.add((uriMotsCles, RDF.type, OWL.Class))
g.add((uriMotsCles, RDFS.label, Literal('MotClé', lang='fr')))

## Définition des objet properties
# fréquence
uriFrequence = URIRef(f'{baseURI}/frequence/')
g.add((uriFrequence, RDF.type, OWL.ObjectProperty))
g.add((uriFrequence, RDFS.label, Literal("Fréquence", lang='fr')))

for record in records:
    # Disciplines
    idDiscipline = record['expertise.disciplines.uid']
    uriDiscipline = URIRef(f'{baseURI}/disciplines/{idDiscipline}')
    
    nomDiscipline = records['expertise.disciplines.nom'] 

    g.add((uriDiscipline, RDF.type, uriDisciplines))
    g.add((uriDiscipline, RDF.type, OWL.Class))
    g.add((uriDiscipline, RDFS.label, Literal(nomDiscipline, lang='fr')))

    # Mots-clés
    idMotCle = records['expertise.motsCles.uid']
    uriMotCle = URIRef(f'{baseURI}/motsCles/{idMotCle}')
    nomMotCle = Literal(records['expertise.motsCles.nom'])

    ### Reprendre ici 
    compte = Literal(kw['expertises.motCles.count'], datatype=XSD.integer)

    # Lien discipline <> mot-clé
    g.add((uri, RDFS.subClassOf, uriDiscipline))

    # Construction du noeud pour le mot-clé
    g.add((uri, RDF.type, OWL.Class))
    g.add((uri, RDFS.label, label))
    g.add((uri, uriCount, compte))

In [ ]:
g.serialize('graph_test_neurosciences.ttl', format='ttl')

In [ ]:
listeDisciplines = disciplines['expertise.disciplines.uid'].tolist()
df = pd.DataFrame()

for discipline in listeDisciplines:
    subdf = disciplines[disciplines['expertise.disciplines.uid'] == discipline]
    subdf = explodeNormalize(subdf, 'expertise.motsCles')
    subdf = subdf.dropna(subset='expertise.motsCles.nom')

    # Pour chaque chercheur, on prend les trois premiers mots-clés fournis pour réduire la taille du graphe
    subdf = subdf[subdf['expertise.motsCles.ordre'].astype(int) <= 3]

    subdf = subdf.drop_duplicates(subset=['idsadvr', 'expertise.motsCles.uid'])
    subdf = subdf[['idsadvr',	'expertise.disciplines.uid', 'expertise.disciplines.nom', 'expertise.motsCles.uid',
                    'expertise.motsCles.nom',	'expertise.motsCles.ordre',	'expertise.motsCles.codeLangue']]

    freqMotsCles = pd.DataFrame(plotVariable(subdf, 'expertise.motsCles.uid')).to_dict('records')
    freqMotsCles = {x['labels'] : x['count'] for x in freqMotsCles}

    subdf['expertises.motCles.count'] = subdf['expertise.motsCles.uid'].map(freqMotsCles)
    subdf = subdf.drop_duplicates(subset = ['expertise.motsCles.uid']).drop(columns='idsadvr')

    subdf = subdf.sort_values(by='expertises.motCles.count', ascending=False).reset_index(drop=True)

    df = pd.concat([df, subdf])